In [1]:
import os
import json

import numpy as np
import pandas as pd

from ipywidgets import Dropdown

from bqplot import Lines, Figure, LinearScale, DateScale, Axis

from ipyleaflet import Map, GeoJSON, WidgetControl

In [2]:
data = pd.read_json(os.path.abspath("nations.json"))

In [3]:
def clean_data(data):
    for column in ["income", "lifeExpectancy", "population"]:
        data = data.drop(data[data[column].apply(len) <= 4].index)
    return data


def extrap_interp(data):
    data = np.array(data)
    x_range = np.arange(1800, 2009, 1.0)
    y_range = np.interp(x_range, data[:, 0], data[:, 1])
    return y_range


def extrap_data(data):
    for column in ["income", "lifeExpectancy", "population"]:
        data[column] = data[column].apply(extrap_interp)
    return data

In [4]:
data = clean_data(data)
data = extrap_data(data)

In [5]:
data

,name,region,income,population,lifeExpectancy
0,Angola,Sub-Saharan Africa,"[359.93, 359.93, 359.93, 359.93, 359.93, 359.9...","[1567028.0, 1567028.0, 1567028.0, 1567028.0, 1...","[26.98, 26.98, 26.98, 26.98, 26.98, 26.98, 26...."
1,Benin,Sub-Saharan Africa,"[553.72, 553.72, 553.72, 553.72, 553.72, 553.7...","[636559.0, 636559.0, 636559.0, 636559.0, 63655...","[31.0, 31.0, 31.0, 31.0, 31.0, 31.0, 31.0, 31...."
2,Botswana,Sub-Saharan Africa,"[407.36, 407.36, 407.36, 407.36, 407.36, 407.3...","[121000.0, 121000.0, 121000.0, 121000.0, 12100...","[33.6, 33.6, 33.6, 33.6, 33.6, 33.6, 33.6, 33...."
3,Burkina Faso,Sub-Saharan Africa,"[454.33, 454.33, 454.33, 454.33, 454.33, 454.3...","[1665421.0, 1665421.0, 1665421.0, 1665421.0, 1...","[29.2, 29.2, 29.2, 29.2, 29.2, 29.2, 29.2, 29...."
4,Burundi,Sub-Saharan Africa,"[447.59, 447.59, 447.59, 447.59, 447.59, 447.5...","[899097.0, 899097.0, 899097.0, 899097.0, 89909...","[31.5, 31.5, 31.5, 31.5, 31.5, 31.5, 31.5, 31...."
...,...,...,...,...,...
174,Thailand,East Asia & Pacific,"[496.98, 496.98, 496.98, 496.98, 496.98, 496.9...","[4665000.0, 4665000.0, 4665000.0, 4665000.0, 4...","[30.4, 30.4, 30.4, 30.4, 30.4, 30.4, 30.4, 30...."
175,Timor-Leste,East Asia & Pacific,"[514.12, 514.3505, 514.581, 514.8115, 515.042,...","[137262.0, 137262.0, 137262.0, 137262.0, 13726...","[28.97, 28.97, 28.97, 28.97, 28.97, 28.97, 28...."
177,Tonga,East Asia & Pacific,"[667.71, 667.71, 667.71, 667.71, 667.71, 667.7...","[18658.0, 18654.325581395347, 18650.6511627907...","[57.91, 57.91, 57.91, 57.91, 57.91, 57.91, 57...."
178,Vietnam,East Asia & Pacific,"[459.71, 459.71, 459.71, 459.71, 459.71, 459.7...","[6551000.0, 6551000.0, 6551000.0, 6551000.0, 6...","[32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32...."


In [6]:
date_start = pd.datetime(1800, 12, 31)
date_end = pd.datetime(2009, 12, 31)

date_scale = DateScale(min=date_start, max=date_end)

/var/folders/d3/jm92yqd96dz8jxjznnxtzjmm0000gp/T/ipykernel_23332/1043072328.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  date_start = pd.datetime(1800, 12, 31)
/var/folders/d3/jm92yqd96dz8jxjznnxtzjmm0000gp/T/ipykernel_23332/1043072328.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  date_end = pd.datetime(2009, 12, 31)


In [7]:
date_data = pd.date_range(start=date_start, end=date_end, freq="A", normalize=True)

In [8]:
country_name = "Angola"
data_name = "income"

x_data = data[data.name == country_name][data_name].values[0]

In [9]:
x_scale = LinearScale()

lines = Lines(x=date_data, y=x_data, scales={"x": date_scale, "y": x_scale})

ax_x = Axis(label="Year", scale=date_scale, num_ticks=10, tick_format="%Y")
ax_y = Axis(
    label=data_name.capitalize(), scale=x_scale, orientation="vertical", side="left"
)

figure = Figure(
    axes=[ax_x, ax_y],
    title=country_name,
    marks=[lines],
    animation_duration=500,
    layout={"max_height": "250px", "max_width": "400px"},
)
figure

Figure(animation_duration=500, axes=[Axis(label='Year', num_ticks=10, scale=DateScale(max=datetime.datetime(20…

In [10]:
def update_figure(country_name, data_name):
    lines.y = data[data.name == country_name][data_name].values[0]
    ax_y.label = data_name.capitalize()
    figure.title = country_name

In [11]:
country_name = "Benin"
data_name = "income"

update_figure(country_name, data_name)

In [12]:
country_name = "Angola"
data_name = "population"

update_figure(country_name, data_name)

In [13]:
with open("./countries.geo.json") as f:
    countries = json.load(f)

In [14]:
m = Map(zoom=3)

geo = GeoJSON(
    data=countries,
    style={"fillColor": "white", "weight": 0.5},
    hover_style={"fillColor": "#1f77b4"},
    name="Countries",
)
m.add_layer(geo)

m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [15]:
widget_control1 = WidgetControl(widget=figure, position="bottomright")

m.add_control(widget_control1)


def on_hover(event, feature, **kwargs):
    global country_name

    country_name = feature["properties"]["name"]
    update_figure(country_name, data_name)
    print("feature",feature)
geo.on_hover(on_hover)

In [16]:
dropdown = Dropdown(
    options=["income", "population", "lifeExpectancy"],
    value=data_name,
    description="Plotting:",
)


def on_click(change):
    global data_name

    data_name = change["new"]
    update_figure(country_name, data_name)


dropdown.observe(on_click, "value")

widget_control2 = WidgetControl(widget=dropdown, position="bottomleft")

m.add_control(widget_control2)